### This case study observes weather data from two sources from 1989 to 2010 of Austin, Texas. Climate measurments for each hour of the day, averaged for over 30 years.

## Loading the Data 

In [ ]:
# Import pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

df = pd.read_csv("../input/noaa-qclcd-2011/2011_Austin_Weather.txt")
df.head()

* We read the file into a DataFrame using the default arguments. 
* After inspecting it, we see that there is no header, and thus the columns have no labels. There is also no obvious index column, since none of the data columns contain a full date or time. 
* We re-read the file specifying that there are no headers supplied. 

In [ ]:
# Read the 2011_Austin_Weather.txt as a DataFrame attributing no header
df = pd.read_csv('../input/noaa-qclcd-2011/2011_Austin_Weather.txt', header=None)
df.head()

In [ ]:
with open('../input/column-label/column_labels.txt') as file:
    column_labels = file.read()
    
# Split on the comma to create a list: column_labels_list    
    column_labels_list = column_labels.split(',')
                 
# Assign the new column labels to the DataFrame: df.columns
    df.columns = column_labels_list

df.head()

### We drop a few columns with attributes that are irrelevant

In [ ]:
list_to_drop = ['sky_conditionFlag',
 'visibilityFlag',
 'wx_and_obst_to_vision',
 'wx_and_obst_to_visionFlag',
 'dry_bulb_farenFlag',
 'dry_bulb_celFlag',
 'wet_bulb_farenFlag',
 'wet_bulb_celFlag',
 'dew_point_farenFlag',
 'dew_point_celFlag',
 'relative_humidityFlag',
 'wind_speedFlag',
 'wind_directionFlag',
 'value_for_wind_character',
 'value_for_wind_characterFlag',
 'station_pressureFlag',
 'pressure_tendencyFlag',
 'pressure_tendency',
 'presschange',
 'presschangeFlag',
 'sea_level_pressureFlag',
 'hourly_precip',
 'hourly_precipFlag',
 'altimeter',
 'record_type',
 'altimeterFlag',
 'junk']

In [ ]:
# Remove the appropriate columns: df_dropped
df_dropped = df.drop(list_to_drop, axis='columns')

# output of df_dropped.head()
print(df_dropped.head())

### Constructing a Time Series Data

We clean up the date and Time columns and combine them into a datetime collection to be used as the Index
* First, we convert the 'date' column to a string with .astype(str) and assign to df_dropped['date']
* Convert the date_string Series to datetime values with pd.to_datetime(). and using df.set_index() to create new time_series index

In [ ]:
# Convert the date column to string: df_dropped['date']
df_dropped['date'] = df_dropped.date.astype(str)
# Pad leading zeros to the Time column: df_dropped['Time']
df_dropped['Time'] = df_dropped['Time'].apply(lambda x:'{:0>4}'.format(x))

# Concatenate the new date and Time columns
date_string = df_dropped['date']+df_dropped['Time']

# Convert the date_string Series to datetime: date_times
date_times = pd.to_datetime(date_string, format='%Y%m%d%H%M')

# Set the index to be the new date_times container: df_clean
df_clean = df_dropped.set_index(date_times)

# Print the output of df_clean.head()
df_clean.head()

### Filling in Missing Values 

In [ ]:
df_clean.info()

###  Checking and specifying missing values ussing parameter errors='coerce' 

In [ ]:
# Print the dry_bulb_faren temperature between 8 AM and 9 AM on June 20, 2011
print(df_clean.loc['2011-06-20 8:00':'2011-06-20 9:00', 'dry_bulb_faren'])

> * The numeric columns contain missing values labeled as 'M' 
* We're going to solve both, missing values and type convertion by using pd.to_numeric()
* We use errors='coerce'  because  whenever a convertion error happens, such as string to float, the values is converted to a NaN, representing a missing value.
* We convert dry_bulb_faren, wind_speed, and dew_point_faren

In [ ]:
# Convert the dry_bulb_faren column to numeric values: df_clean['dry_bulb_faren']
df_clean['dry_bulb_faren'] = pd.to_numeric(df_clean['dry_bulb_faren'], errors='coerce')

# Print the transformed dry_bulb_faren temperature between 8 AM and 9 AM on June 20, 2011
print(df_clean.loc['2011-06-20 8:00':'2011-06-20 9:00', 'dry_bulb_faren'])

# Convert the wind_speed and dew_point_faren columns to numeric values
df_clean['wind_speed'] = pd.to_numeric(df_clean['wind_speed'], errors='coerce')
df_clean['dew_point_faren'] = pd.to_numeric(df_clean['dew_point_faren'], errors='coerce')

### Statistical EDA

* Partial DateTime selection

In [ ]:
# Print the median of the dry_bulb_faren column
print(df_clean.dry_bulb_faren.median())

# Print the median of the dry_bulb_faren column for the time range '2011-Apr':'2011-Jun'
print(df_clean.loc['2011-Apr':'2011-Jun', 'dry_bulb_faren'].median())

# Print the median of the dry_bulb_faren column for the month of January
print(df_clean.loc['2011-Jan', 'dry_bulb_faren'].median())

> * We now compare the 2011 weather data with the 30-year normals reported in 2010. We find outm how much hotter was every day in 2011 than expected from the 30-year averag
* Downsample df_clean with daily frequency and aggregate by the mean. 
* Extract the dry_bulb_faren column from daily_mean_2011
* We see that the indexes of df_clean and df_climate are not aligned - df_clean has dates in 2011, while df_climate has dates in 2010. We make use of of *.reset_index()* method to make sure the Series align properly.

#### Compute Daily Frequency Average 

In [ ]:
# Read the file from input
df_climate_2010 = pd.read_csv('../input/weather_data_austin_2010/weather_data_austin_2010.csv')

### We downsample the index from hourly to daily based

We prepare the data from 2010-austin-weather data set. 
* First we convert it to time-series format

In [ ]:
#print(df_climate_2010)
# set.index() to df_climate_2010 for time series 
df_climate_2010.Date = pd.to_datetime(df_climate_2010.Date)
df_climate_2010.set_index(df_climate_2010.Date, inplace=True)
df_climate_2010_copy = df_climate_2010.copy()
df_climate_2010.head(2)

In [ ]:
# Downsample df_clean by day and aggregate by mean: daily_mean_2011
daily_mean_2011 = df_clean.resample('D').mean()

# Extract the dry_bulb_faren column from daily_mean_2011 using .values: daily_temp_2011
daily_temp_2011 = daily_mean_2011['dry_bulb_faren'].values

# Downsample df_climate by day and aggregate by mean: daily_climate
daily_climate_2010 = df_climate_2010.resample('D').mean()

# Extract the Temperature column from daily_climate using .reset_index(): daily_temp_climate
daily_temp_climate = daily_climate_2010.reset_index()['Temperature']

>### We compare and see the difference in Temperatures between 2010 and 2011

In [ ]:

# Compute the difference between the two arrays and print the mean difference
difference = daily_temp_2011 - daily_temp_climate
print(difference.mean())

#### As computed above the average daily temperature was **1.33** higher between 2011 and 2010.

>### On average, how much hotter is it when the sun is shining?

 We get this information with  'sky_condition' which provides information about whether the day was sunny ('CLR') or overcast ('OVC').

In [ ]:
#print(df_clean)

In [ ]:
# Select days that are sunny: sunny
sunny = df_clean.loc[df_clean['sky_condition'].str.contains('CLR')]

# Select days that are overcast: overcast
overcast = df_clean.loc[df_clean['sky_condition'].str.contains('OVC')]

# Resample sunny and overcast, aggregating by maximum daily temperature
sunny_daily_max = sunny.resample('D').max()
overcast_daily_max = overcast.resample('D').max()

# Print the difference between the mean of sunny_daily_max and overcast_daily_max
print(sunny_daily_max.mean() - overcast_daily_max.mean())

 * We see here that in 2011 sunny days were *6.5 * hotter than in 2010.

> **Is there a correlation between temperature and visibility?**

 * We select the 'visibility' and 'dry_bulb_faren' columns and resample them by week and aggregate the mean
 * We use the Pearson correlation to compute the coefficient. The values close to 1 here would indicate that there is a strong correlation between temperature and visibility.
 * A value close to 1 here would indicate that there is a strong correlation between temperature and visibility.

In [ ]:
# Select the visibility and dry_bulb_faren columns and resample them: weekly_mean
visibility_temperature=df_clean[['visibility', 'dry_bulb_faren']]
weekly_mean = visibility_temperature.resample('W').mean()
# Print the output of weekly_mean.corr()
print(weekly_mean.corr())

**We are not able to see the visibilty column. Let us look into the column some more**

In [ ]:
df_clean.info()

We convert the visibilty column from type object to numeric.

In [ ]:
df_clean['visibility'] = pd.to_numeric(df_clean['visibility'], errors='coerce')
visibility_temperature=df_clean[['visibility', 'dry_bulb_faren']]
weekly_mean = visibility_temperature.resample('W').mean()
# Print the output of weekly_mean.corr()
print(weekly_mean.corr())

We see a medium correlation of **0.49**, a medium correlation. 

In [ ]:
# Plot weekly_mean with subplots=True
weekly_mean.plot(subplots=True)
plt.show()

> ### Fraction of days that are sunny?

In [ ]:
#a Boolean Series for sunny days: sunny
sunny = df_clean['sky_condition'] == 'CLR'
# Resample the Boolean Series by day and compute the sum: sunny_hours
sunny_hours = sunny.resample('D').sum()

# Resample the Boolean Series by day and compute the count: total_hours
total_hours = sunny.resample('D').count()

# Divide sunny_hours by total_hours: sunny_fraction
sunny_fraction = sunny_hours / total_hours

# Make a box plot of sunny_fraction
sunny_fraction.plot(kind='box')
plt.show()

The median for sunny days is around **18%**.  75% of the values are under 40% of the sunny days.

> ### What is the maximum temperature and dew point of each month?

*  We  resample the 'dew_point_faren' and 'dry_bulb_faren' to get the maximum temperature and dew point in each month
* Dew point is a measure of relative humidity based on pressure and temperature. A dew point above 65 is considered uncomfortable while a temperature above 90 is also considered uncomfortable.
* We generate a histogram to get a better idea 

In [ ]:
# Resample dew_point_faren and dry_bulb_faren by Month, aggregating the maximum values: monthly_max
monthly_max = df_clean[['dew_point_faren', 'dry_bulb_faren']].resample('M').max()

# Generate a histogram with bins=8, alpha=0.5, subplots=True
monthly_max.plot(kind='hist', bins=8, alpha=0.5, subplots=True)

# Show the plot
plt.show()

####  We see that the maximum dew point is above 65 every month 

> ### What is the Probability of high temperatures in 2011?

* We compare the maximum temperature in August 2011 against that of the August 2010 climate normals. 
* We use a CDF plot to determine the probability of the 2011 daily maximum temperature in August being above the 2010 climate normal value.
* We  select the maximum temperature in August in df_climate, and then maximum daily temperatures in August 2011. The days are then filtered out in August 2011 that were above the August 2010 maximum, and then used to construct a CDF plot. 
 

In [ ]:
# Extract the maximum temperature in August 2010 from df_climate: august_max
august_max = df_climate_2010_copy.loc['2010-08', 'Temperature'].max()
print('Max temperature registered in August 2010 was ' + str(august_max.max()))

# Resample the August 2011 temperatures in df_clean by day and aggregate the maximum value: august_2011
august_2011 = df_clean.loc['2011-Aug', 'dry_bulb_faren'].resample('D').max()
print('Max temperature registered in August 2011 was ' + str(august_2011.max()))

# Filter out days in august_2011 where the value exceeded august_max: august_2011_high
august_2011_high = august_2011.loc[august_2011 > august_max]


# Construct a CDF of august_2011_high
august_2011_high.plot(kind='hist', normed=True,cumulative=True, bins=25, linestyle='-', title='Probability of hotter day in August 2011')
plt.xlabel('Registered Temperature')

# Display the plot
plt.show()

###  It shows that there was a 50% probability of the 2011 daily maximum temperature in August being 5 degrees above the 2010 climate normal value!

## Conclusion

  ** This dataset is my approach to using the Time Series Capabilites offered by Pandas.  
     With the help of <a href="https://campus.datacamp.com/courses/pandas-foundations/">Datacamp's</a> Pandas Foundation course, 
     I was able to  perform statistical and visual EDA  in pandas. **